In [9]:
from datetime import date
from dateutil.parser import parse
import email
import mailbox
import numpy as np
import os
import re
import sys
from time import asctime

In [4]:
import pandas as pd

In [28]:
DIRNAME = os.path.dirname(os.getcwd())+os.sep+'enron'+os.sep
DATA = DIRNAME+'enron_mail'+os.sep
MBOX = DATA+'enron.mbox'

In [4]:
def maildir_to_mailbox(MAILDIR):
    with open(MBOX, 'w') as mbox:   
        for (root, dirs, file_names) in os.walk(MAILDIR):
            if root.split(os.sep)[-1].lower() != 'inbox':
                continue

            # Process each message in 'inbox'
            for file_name in file_names:
                file_path = os.path.join(root, file_name)
                message_text = open(file_path).read()

                # Compute fields for the From_ line in a traditional mbox message

                #_from = re.search(r"From: ([^\r]+)", message_text).groups()[0]
                #_date = re.search(r"Date: ([^\r]+)", message_text).groups()[0]

                # Convert _date to the asctime representation for the From_ line

                #_date = asctime(parse(_date).timetuple())

                msg = email.message_from_string(message_text)
                #msg.set_unixfrom('From %s %s' % (_from, _date))

                mbox.write(msg.as_string(unixfrom=True) + "\n\n")

    return mbox


In [5]:
maildir_to_mailbox(DIRNAME+'enron'+os.sep+'enron_mail'+os.sep+'maildir'+os.sep)

<_io.TextIOWrapper name='c:\\Users\\utilisateur\\Desktop\\Projets\\enron\\enron_mail\\enron.mbox' mode='w' encoding='cp1252'>

In [29]:
def mbox_to_df():
    """Convert the mailbox to a dataframe.

    Returns:
        dict: Enron corpus sample dataframe

    """
    mbox = mailbox.mbox(MBOX)

    mbox_dict = {}

    for i, msg in enumerate(mbox):
        mbox_dict[i] = {}
        for header in msg.keys():
            mbox_dict[i][header] = msg[header]

        mbox_dict[i]['Body'] = msg.get_payload().replace('\n', ' ').replace('\t', ' ').strip()

    df = pd.DataFrame.from_dict(mbox_dict, orient='index')
    # TODO: add field before bankrupt(df):
    # bankrupt = date(2001, 12, 2) bool
    # df.index = df['Date'].apply(pd.to_datetime)

    # df = data_cleaning(df)
    # df.to_csv(DATA+'enron_mail.csv', sep='|')
    # sample_df = df.sample(200)
    df.to_csv(DATA+'sample_enron_mail.csv', sep='|')

    return df

In [ ]:
def data_cleaning(df):
    """Clean the dataframe.

    Args:
        df (dict): dictionnary to clean

    Returns:
        dict: Enron corpus cleaned dataframe

    """
    cols_to_keep = ['Date', 'From', 'To', 'Cc', 'Bcc', 'Subject', 'Body']
    df = df[cols_to_keep]

    df = df.drop_duplicates()
    df = df.drop_duplicates(subset=['Date', 'From', 'To'])

    return df

In [6]:
sample_df = pd.read_csv('enron_mail/sample_enron_mail.csv', sep='|')
df = data_cleaning(df)